In [1]:
# main.py
import torch
from torch.utils.data import DataLoader
from dataloaderv2 import EchoVideoDataset
from modelv5 import MobileNetV3UNet
from utils import train_model  # Import the training function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize datasets
train_dataset = EchoVideoDataset(root="./data/echodynamic", split='train')
val_dataset = EchoVideoDataset(root="./data/echodynamic", split='val')

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Initialize the model with 4 channels and backbone = True (for pretrained MobileNetV3)
# Do NOT move to device yet.
model = MobileNetV3UNet(in_channels=4, out_channels=1, config_name="large", backbone=True)

# Load pretrained mobilenet_v3_large weights and adapt first layer
pretrained_mobilenet = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v3_large', pretrained=True)
pretrained_state = pretrained_mobilenet.state_dict()

old_weight = pretrained_state["features.0.0.weight"]  # (out_channels, 3, kh, kw)
out_channels, in_channels, kh, kw = old_weight.shape
new_weight = torch.zeros((out_channels, 4, kh, kw))
new_weight[:, :3, :, :] = old_weight
new_weight[:, 3, :, :] = old_weight[:, 0, :, :]

model.encoder.backbone_model.features[0][0].weight = torch.nn.Parameter(new_weight)

# Adapt the rest of the weights
adapted_state = model.state_dict()
for k in pretrained_state:
    if k == "features.0.0.weight":
        continue
    if k in adapted_state and adapted_state[k].shape == pretrained_state[k].shape:
        adapted_state[k] = pretrained_state[k]

model.load_state_dict(adapted_state)

# Now move model to GPU/CPU as per device
model = model.to(device)

# Train the model
model = train_model(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4)


/home/hc4293/miniconda3/envs/imgsenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hc4293/miniconda3/envs/imgsenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /home/hc4293/.cache/torch/hub/pytorch_vision_v0.10.0
Epoch 1/10:   0%|          | 0/7465 [00:00<?, ?batch/s]/home/hc4293/echoframe/dataloaderv2.py:109: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle modul

Epoch 1/10, Train Loss: -0.9571, Val Loss: -0.9867


Epoch 2/10:  57%|█████▋    | 4260/7465 [10:21<07:47,  6.85batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacity of 15.72 GiB of which 227.44 MiB is free. Including non-PyTorch memory, this process has 15.50 GiB memory in use. Of the allocated memory 14.17 GiB is allocated by PyTorch, and 1.14 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)